# Tokenizers
* In NLP most data is raw text, but ML can't read or understand in raw form
    * Only Work with numbers
* Tokenizer translates text to numbers
* Several approaches - Objective is to find most meaningful representation:
## Word-Based
* ![Image](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/word_based_tokenization.svg)
* There're different ways to split text. For example, using whitespace to tokenize text into words by applying python's split function:
```python
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)
```
Would output:
```['Jim', 'Henson', 'was', 'a', 'puppeteer']```
* Variations of WTs w/ extra punctuation rules.
    * Can have large "vocabularies", w/ vocab defined by total number of independent tokens we have in our corpus.
    * Each word assigned ID, from 0-(vocabsize). Model uses IDs to indentify each word
    * The problem:
        * There're >500k words in English language, lots of tokens
        * Dog & Dogs are similar, but model wouldn't be able to relate them
        * We need custom token to represent words not in vocab, known as "unknown" token (often represented as "UNK" or "<unk>")
        * Goal is to have **AS FEW WORDS AS POSSIBLE UNK**
        * Which is why we use a character based tokenizer
## Character-Based
* CB splits chars rather than words, w/ two main benifits:
    * Smaller vocab
    * Fewer out-of-vocab tokens since you can build all words
* Also not perfect: Problems w/ spaces & punctuation
    * Less meaningful than words, but language dependent
        * In Chinese, each character carries more info than latin language
    * We'll end up w/ very large amt of tokenz to be processed by model
        * WB = 1 Token, CB = 10 Tokens
        * Thus, Subword
## Subword-Based
* Freq. used words not split, but rare words should be
* E.g. "Annoyingly" is rare, thus, ```["Annoying", "ly"]```, as both will show as frequent standalone subwords, while the meaning is kept by the composite meaning of annoyingly.
* E.g. "Tokenization"
    * BERT writes ```['token','##ization'] where "##" shows that it's PART of the word